In [1]:
import pandas as pd
import numpy as np

In [2]:
from gp_pref_elicit_luisa import gaussian_process as GP 
from gp_pref_elicit_luisa import dataset as data 
from gp_pref_elicit_luisa import acquisition_function as acquisition_function
from gp_pref_elicit_luisa.gp_utilities import utils_ccs as gp_utils_ccs
from gp_pref_elicit_luisa.gp_utilities import utils_data as gp_utils_data
from gp_pref_elicit_luisa.gp_utilities import utils_user as gp_utils_users

In [3]:
# generating synthetic dataset which outputs value vectors
# gp_utils_ccs.get_ccs(2, 20)
# outputs a synthetic Pareto Coverage Set of value vectors with 2 objectives and 20 datapoints

The original dataset, in the data_preprocessed.ipynb file, can not be used for the Gaussian Process as it is not in the form of value vectors. To bring it into the form of value vectors, the dataset needs to be passed into a Multi-Objective Shortest Path Planning Solver which will output value vectors. These value vectors can then be an input for the GP. Thus, for now, I am using a synthetic Pareto Coverage Set (PCS) with 2 objectives and 20 datapoints which gives us synthetic value vectors for input to the GP.

In [4]:
synthetic_pcs = np.array([[0.08354893, 0.99771414],
       [0.17329006, 0.98281369],
       [0.93898169, 0.37256163],
       [0.57360241, 0.848919  ],
       [0.98568813, 0.18298825],
       [0.11718526, 0.99496982],
       [0.69436633, 0.7426556 ],
       [0.419793  , 0.90681482],
       [0.99850198, 0.05548771],
       [0.97244122, 0.25252888],
       [0.97837106, 0.22331134],
       [0.21266473, 0.97269664],
       [0.8964366 , 0.49411589],
       [0.25476081, 0.96052451],
       [1.        , 0.        ],
       [0.35559809, 0.92879935],
       [0.80235668, 0.62389126],
       [0.        , 1.        ],
       [0.27334299, 0.95488636],
       [0.99132009, 0.1452147 ]])

# to generate new dataset: uncomment the line below and comment the dataset above
# synthetic_pcs = gp_utils_ccs.get_ccs(num_objectives=2, ccs_size=20) #feel free to modify the number of objectives


In [5]:
# initializing GP, Dataset and Acquisition Function 
GP = GP.GPPairwise(num_objectives=2)
utils_comparisons = data.DatasetPairwise(num_objectives=2)
acquisition_function_DA = acquisition_function.DiscreteAcquirer(input_domain=synthetic_pcs, query_type='pairwise', seed=None, acquisition_type='expected improvement')
acquisition_function_EI = acquisition_function.get_expected_improvement(datapoints=synthetic_pcs, gaussian_process=GP, datapoints_hist=acquisition_function_DA.history, xi=0.01)
# getting the user preferences for generating a ground truth utility function
user_pref = gp_utils_users.UserPreference(num_objectives=2, std_noise=0.1)

In [6]:
ground_truth_utility_function_dataset = user_pref.get_preference(synthetic_pcs, add_noise=True)
ground_truth_utility_function_dataset = np.max(ground_truth_utility_function_dataset)
ground_truth_utility_function_dataset

0.7259590197414958

In [7]:
# starting points in the dataset
start_points = acquisition_function_DA.get_start_points(gaussian_process=GP) 
# generating ground truth utility function for the starting points
ground_truth_utility_function_start = user_pref.get_preference(start_points, add_noise=True)
# getting the index of the highest utility
highest_utility_index_start = np.argmax(ground_truth_utility_function_start)
# getting the index of the lowest utility
lowest_utility_index_start = np.argmin(ground_truth_utility_function_start)
# mapping the indices of the highest utility to the actual datapoint in the dataset
highest_utility_point_dataset = start_points[highest_utility_index_start]
# mapping the indices of the lowest utility to the actual datapoint in the dataset
lowest_utility_point_dataset = start_points[lowest_utility_index_start]

# adding the highest utility point, which is the winner, to the dataset 
utils_comparisons.add_single_comparison(highest_utility_point_dataset, lowest_utility_point_dataset)

# adding the actual point to the array 
current_max_points = highest_utility_point_dataset

# updating the GP
GP.update(utils_comparisons)

# checking which points to exclude
exclude_points = acquisition_function.exclude_points_pairwise(dataset=utils_comparisons)

print('Start points: ', start_points, '\n',
      'Point in the dataset having highest utility: ', highest_utility_point_dataset, '\n',
      'Point in the dataset having lowest utility: ', lowest_utility_point_dataset, '\n',
      'Current max: ', current_max_points, '\n', 
      'Excluded points: ', exclude_points)

Start points:  (array([0.08354893, 0.99771414]), array([0.21266473, 0.97269664])) 
 Point in the dataset having highest utility:  [0.21266473 0.97269664] 
 Point in the dataset having lowest utility:  [0.08354893 0.99771414] 
 Current max:  [0.21266473 0.97269664] 
 Excluded points:  [[0.21266473 0.97269664]
 [0.08354893 0.99771414]]


We have 2 stopping conditions:
1. If the next point found according to Expected Improvement is already in the list of compared points (the points which we exclude for comparison - excluded points) then the loop breaks. This is because the loop has explored all the points available to it.
2. If we have a large dataset where the above condition might not always be true then we look at the probability of improvement between points. If the probability of improvement is below 5% (0.05) then the loop terminates because there is no further scope of improvement.

In [8]:
# initializing a threshold
threshold = 0.05
stop_condition = False

while not stop_condition:
  # next points in the dataset according to Expected Improvement
  next_point_EI = acquisition_function_DA.get_next_point_EI(gaussian_process=GP, exclude=exclude_points) 

  # applying first stopping condition
  if next_point_EI.tolist() in exclude_points.tolist():
    break

  # generating ground truth utility function for the next points
  ground_truth_utility_function_next_EI = user_pref.get_preference(next_point_EI, add_noise=True)

  # generating ground truth utility function for current best point
  ground_truth_utility_function_current_best = user_pref.get_preference(current_max_points, add_noise=True)

  # adding the highest utility point, which is the winner, to the dataset only if the current point being evaluated according to 
  # expected improvement has a greater utility than the utility of the current best point evaluated previously
  if ground_truth_utility_function_next_EI > ground_truth_utility_function_current_best:
    utils_comparisons.add_single_comparison(next_point_EI, current_max_points)
    current_max_points = next_point_EI # we update the current best point 
  else:
    utils_comparisons.add_single_comparison(current_max_points, next_point_EI)
  
  # computing the datapoints for this comparisons dataset
  utility_comparisons_datapoints_next_EI = utils_comparisons.datapoints

  # updating the GP
  GP.update(utils_comparisons)

  # checking which points to exclude
  exclude_points = acquisition_function.exclude_points_pairwise(dataset=utils_comparisons)
  exclude_points = np.append(exclude_points, [current_max_points], axis=0)

  # difference between actual best utility and observed best utility
  regret = np.subtract(ground_truth_utility_function_dataset, ground_truth_utility_function_current_best)

  # calculating the probability of improvement
  prob_imprv = acquisition_function.get_probability_of_improvement(x=next_point_EI, gaussian_process=GP, x_previous=current_max_points)

  print('Excluded points: ', exclude_points, '\n',
    'next points for the GP acc to EI: ', next_point_EI, '\n',
    'Comparison dataset datapoints: ', utility_comparisons_datapoints_next_EI, '\n',
    'Current max: ', current_max_points, '\n', 
    'Ground truth utility of the current max: ', ground_truth_utility_function_current_best, '\n',
    'Excluded points: ', len(exclude_points), '\n',
    'Simple Regret: ', regret, '\n',
    'Probability of improvement: ', prob_imprv)
  
  # applying second stopping condition
  if prob_imprv < threshold:
    stop_condition = True

Excluded points:  [[0.17329006 0.98281369]
 [0.21266473 0.97269664]
 [0.17329006 0.98281369]] 
 next points for the GP acc to EI:  [0.17329006 0.98281369] 
 Comparison dataset datapoints:  [[0.21266473 0.97269664]
 [0.08354893 0.99771414]
 [0.17329006 0.98281369]] 
 Current max:  [0.17329006 0.98281369] 
 Ground truth utility of the current max:  [0.42441169] 
 Excluded points:  3 
 Simple Regret:  [0.30154733] 
 Probability of improvement:  [0.49135346]
Excluded points:  [[0.17329006 0.98281369]
 [0.21266473 0.97269664]
 [1.         0.        ]
 [0.17329006 0.98281369]] 
 next points for the GP acc to EI:  [1. 0.] 
 Comparison dataset datapoints:  [[0.21266473 0.97269664]
 [0.08354893 0.99771414]
 [0.17329006 0.98281369]
 [1.         0.        ]] 
 Current max:  [0.17329006 0.98281369] 
 Ground truth utility of the current max:  [0.58477144] 
 Excluded points:  4 
 Simple Regret:  [0.14118758] 
 Probability of improvement:  [0.38603361]
Excluded points:  [[0.99850198 0.05548771]
 [0.1